In [1]:
import os
import re
import pickle
import pandas as pd
import numpy as np
import seaborn as sn
import keras
from keras.preprocessing import text, sequence
from keras.layers import Input, Dense, Embedding, Flatten, Conv2D, MaxPool2D, Bidirectional, LSTM, GRU, concatenate, GlobalMaxPooling1D, GlobalAveragePooling1D, SpatialDropout1D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.models import Model, Sequential
from keras import backend as K
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='loss', patience=10)#, verbose=1)
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
lb_enc = LabelEncoder()
import nltk
import string 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data_train = pd.read_excel('/kaggle/input/spamreviewdetection/data_train.xlsx')
data_dev = pd.read_excel('/kaggle/input/spamreviewdetection/data_dev.xlsx')
data_test = pd.read_excel('/kaggle/input/spamreviewdetection/data_test.xlsx')

In [3]:
y_train = lb_enc.fit_transform(data_train['Label'])

y_dev = lb_enc.fit_transform(data_dev['Label'])

y_test = lb_enc.fit_transform(data_test['Label'])

In [4]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(data_train['transformed_text'])
tokenizer.fit_on_texts(data_dev['transformed_text'])
# tokenizer.fit_on_texts(data_test['transformed_text'])

text_to_sequence_train = tokenizer.texts_to_sequences(data_train['transformed_text']) 
text_to_sequence_dev = tokenizer.texts_to_sequences(data_dev['transformed_text']) 
text_to_sequence_test = tokenizer.texts_to_sequences(data_test['transformed_text']) 
# text_to_sequence_dev

In [5]:
max_length_sequence_train = max([len(i) for i in text_to_sequence_train])
 
padded_train = pad_sequences(text_to_sequence_train, maxlen=max_length_sequence_train, 
                                    padding = "pre") 
# max_length_sequence_train
padded_train

array([[   0,    0,    0, ...,    5,   52, 1666],
       [   0,    0,    0, ...,  620, 4802, 4803],
       [   0,    0,    0, ...,    0,    0, 6646],
       ...,
       [   0,    0,    0, ...,    1,  113,  143],
       [   0,    0,    0, ...,   14,  119,  618],
       [   0,    0,    0, ...,    7,    5,   21]], dtype=int32)

In [6]:
# max_length_sequence_dev = max([len(i) for i in text_to_sequence_dev])
 
padded_dev = pad_sequences(text_to_sequence_dev, maxlen=max_length_sequence_train, 
                                    padding = "pre") 
len(padded_dev)


1580

In [7]:
# max_length_sequence_test = max([len(i) for i in text_to_sequence_test])
 
padded_test = pad_sequences(text_to_sequence_test, maxlen=max_length_sequence_train, 
                                    padding = "pre") 
len(padded_test)

3938

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Flatten, SimpleRNN, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [9]:
len(tokenizer.word_index)+1

15874

In [10]:
max_length_sequence_train

184

In [11]:
VOC_SIZE = len(tokenizer.word_index) + 1  # 18359

def create_model():
    model = Sequential()
    model.add(Embedding(VOC_SIZE, 250, input_length=max_length_sequence_train))
    model.add(Dropout(0.5))

    model.add(LSTM(96, activation='tanh', recurrent_activation='sigmoid', return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(units=128, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(units=1, activation='sigmoid'))

    model.add(Dense(1, activation='sigmoid'))
    return model

lstm_model = create_model()
lstm_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 184, 250)          3968500   
                                                                 
 dropout (Dropout)           (None, 184, 250)          0         
                                                                 
 lstm (LSTM)                 (None, 96)                133248    
                                                                 
 dropout_1 (Dropout)         (None, 96)                0         
                                                                 
 dense (Dense)               (None, 128)               12416     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1

In [12]:
early_stop = EarlyStopping(monitor='val_loss', patience=3)


In [13]:
history = lstm_model.fit(padded_train, np.reshape(y_train, (-1, 1)), epochs=40, batch_size=32, 
                        validation_data=(padded_dev, np.reshape(y_dev, (-1, 1))), callbacks = [early_stop])

Epoch 1/40
443/443 [==============================] - 41s 74ms/step - loss: 0.6539 - accuracy: 0.7310 - val_loss: 0.6248 - val_accuracy: 0.7316
Epoch 2/40
443/443 [==============================] - 12s 27ms/step - loss: 0.6001 - accuracy: 0.7462 - val_loss: 0.5948 - val_accuracy: 0.7316
Epoch 3/40
443/443 [==============================] - 9s 20ms/step - loss: 0.5776 - accuracy: 0.7462 - val_loss: 0.5843 - val_accuracy: 0.7316
Epoch 4/40
443/443 [==============================] - 10s 22ms/step - loss: 0.5695 - accuracy: 0.7462 - val_loss: 0.5818 - val_accuracy: 0.7316
Epoch 5/40
443/443 [==============================] - 8s 19ms/step - loss: 0.5672 - accuracy: 0.7462 - val_loss: 0.5817 - val_accuracy: 0.7316
Epoch 6/40
443/443 [==============================] - 8s 19ms/step - loss: 0.5666 - accuracy: 0.7462 - val_loss: 0.5819 - val_accuracy: 0.7316
Epoch 7/40
443/443 [==============================] - 8s 17ms/step - loss: 0.5665 - accuracy: 0.7462 - val_loss: 0.5820 - val_accuracy: 0.7

In [18]:
train_loss, train_accuracy = lstm_model.evaluate(padded_train, np.reshape(y_train, (-1, 1)))
print("Train Loss:", train_loss)
print("Train Accuracy:", train_accuracy)

# Evaluate the model on the validation set
val_loss, val_accuracy = lstm_model.evaluate(padded_dev, np.reshape(y_dev, (-1, 1)))
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

# Make predictions on the validation set
y_pred_val = lstm_model.predict(padded_dev)
y_pred_classes_val = np.round(y_pred_val)
f1_val = f1_score(np.reshape(y_dev, (-1, 1)), y_pred_classes_val)
f1_macro_val = f1_score(np.reshape(y_dev, (-1, 1)), y_pred_classes_val, average='macro')
print("F1 Score (val):", f1_val)
print("F1 Score (macro) (val):", f1_macro_val)
cm = confusion_matrix(np.reshape(y_dev, (-1, 1)), y_pred_classes_val)
print("Confusion Matrix: \n", cm)

443/443 [==============================] - 3s 6ms/step - loss: 0.5665 - accuracy: 0.7462
Train Loss: 0.5664530396461487
Train Accuracy: 0.7462212443351746
50/50 [==============================] - 0s 6ms/step - loss: 0.5821 - accuracy: 0.7316
Validation Loss: 0.582076370716095
Validation Accuracy: 0.7316455841064453
50/50 [==============================] - 0s 5ms/step
F1 Score (val): 0.0
F1 Score (macro) (val): 0.42251461988304095
Confusion Matrix: 
 [[1156    0]
 [ 424    0]]


In [20]:
y_pred_test = lstm_model.predict(padded_test)
y_pred_classes_test = np.round(y_pred_test)
# 
# Evaluate performance on test set
acc = accuracy_score(y_test, y_pred_classes_test)
f1 = f1_score(y_test, y_pred_classes_test)
cm = confusion_matrix(y_test, y_pred_classes_test)
print("Accuracy: ", acc)
print("F1 Score: ", f1)
print("Confusion Matrix: \n", cm)
f1_macro = f1_score(y_test, y_pred_classes_test, average='macro')
print("F1 Score (macro): ", f1_macro)

124/124 [==============================] - 1s 5ms/step
Accuracy:  0.7280345352971052
F1 Score:  0.0
Confusion Matrix: 
 [[2867    0]
 [1071    0]]
F1 Score (macro):  0.4213078618662748
